In [60]:
import json
import csv
import pandas as pd
import datetime
import re
from tqdm import tqdm 
from textblob import TextBlob
import emoji

from nltk.corpus import stopwords
from nltk.sentiment.util import *
from nltk import tokenize
stopwords = stopwords.words('english')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

## Lettura CSV

In [62]:
gsw = pd.read_csv("C:\\Users\\fedem\\Desktop\\Università\\DATA_SCIENCE\\I_ANNO\\DATA_MANAGEMENT_AND_DATA_VISUALIZATION\\Progetto\\tweet_gsw.csv", engine = "python", encoding = "utf-8")
tor = pd.read_csv("C:\\Users\\fedem\\Desktop\\Università\\DATA_SCIENCE\\I_ANNO\\DATA_MANAGEMENT_AND_DATA_VISUALIZATION\\Progetto\\tweet_toronto.csv", engine = "python", encoding = "utf-8")
neutro = pd.read_csv("C:\\Users\\fedem\\Desktop\\Università\\DATA_SCIENCE\\I_ANNO\\DATA_MANAGEMENT_AND_DATA_VISUALIZATION\\Progetto\\tweet_neutro.csv", engine = "python", encoding = "utf-8")

## Preprocessing

In [63]:
def is_emoji(s):
    return s in emoji.UNICODE_EMOJI

def tweets_cleaner(text):
    text = re.sub(r'@[A-Za-z0-9_]+',"",text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[0-9]*", "", text)
    text = re.sub(r"(”|“|-|\+|`|#|,|;|\|)*", "", text)
    text = re.sub(r"&amp", "", text)
    return text


def tokenization(sentence):
    
    total_token_ls = []

    token_ls = []
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in stopwords:
            token_ls.append(token)
    total_token_ls.append(token_ls)

    return token_ls

In [66]:
def back_to_clean_sent(token_ls):
    clean_sent = ""
    for word in token_ls:
        clean_sent += (word + " ")
    result = ''
    for char in clean_sent:
        if is_emoji(char):
            result += ' '
        result += char
    return result.strip()

## Prove per capire se è meglio textblob o vader

In [64]:
prova = gsw[["id", "text"]].head(100)

In [67]:
df_gsw = pd.DataFrame(prova["text"])
df_gsw.columns = ["Tweets"]

df_tweet = [0]*len(df_gsw["Tweets"])
token_ls = [0]*len(df_gsw["Tweets"])

for num, text in tqdm(enumerate(df_gsw["Tweets"])):
    token_ls[num] = tokenization(tweets_cleaner(text))

100it [00:00, 2273.99it/s]


In [68]:
sentence_tokenized = [0]*len(df_gsw["Tweets"])
for num, token in tqdm(enumerate(token_ls)):
    sentence_tokenized[num] = back_to_clean_sent(token)

100it [00:00, 25003.30it/s]


In [71]:
# Vader
def sentiment_analysis(sentence):
    ris = analyser.polarity_scores(sentence)["compound"]
    return ris
                             
sentiment_token = []
for x in tqdm(sentence_tokenized):
    sentiment_token.append(sentiment_analysis(x))

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 4347.33it/s]


In [70]:
# Textblob
def getpolarity(text):
    analisi = TextBlob(text)
    polarity = analisi.sentiment.polarity
    return polarity 
    
sentiment_token_blob = []
for x in tqdm(sentence_tokenized):
    sentiment_token_blob.append(getpolarity(x))

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1814.47it/s]


In [72]:
prova["vader"] = sentiment_token
prova["blob"] = sentiment_token_blob

In [73]:
prova


## Funziona meglio vader, utilizzeremo il compound calcolato come misura del sentiment

,id,text,vader,blob
0,1134248118231416833,Let’s fucking go #DubNation,0.0000,-0.600000
1,1134248122291560448,Draymond Green has a 69.2% chance to see more rebounds than any player during the #NBAFinals - he also has a 68.3% chance to see more assists than any other player. \n\nHe has a 12.5% chance to win the Finals MVP Award as well\n\n#StrengthInNumbers \n#NBAPlayoffs,0.9246,0.300000
2,1134248146668851200,"Steph Curry walks into Scotiabank Arena ahead of @warriors/@Raptors Game 1! #StrengthInNumbers \n\n🚨: GSW/TOR, Game 1\n⏰: 9:00pm/et \n🇺🇸: ABC 🇨🇦: Sportsnet https://t.co/NttQJq3NGJ",0.0000,-0.450000
3,1134248155149754369,"Steph Curry walks into Scotiabank Arena ahead of @warriors/@Raptors Game 1! #StrengthInNumbers \n\n🚨: GSW/TOR, Game 1\n⏰: 9:00pm/et \n🇺🇸: ABC 🇨🇦: Sportsnet https://t.co/NttQJq3NGJ",0.0000,-0.450000
4,1134248161931751425,I'm shimmying for #TeamSteph! Do you think Steph can steal the show vs Kawhi? Choose your team for a reminder to watch Game 1 during the NBA Finals on ABC at 9 PM ET. @ESPNNBA #NBAFinals #NBAPlayoffs #DubNation,-0.5411,-0.400000
5,1134248169674485760,"STEPHEN CURRY: ORIGIN STORY\n\n""It's like a dream come true...""\n\nGame 1 of the #NBAFinals presented by @YouTubeTV tips TONIGHT at 9pm/et on ABC &amp; Sportsnet! #StrengthInNumbers https://t.co/nQzHMDePcT",0.7644,-0.075000
6,1134248216604467200,The legend of the Warriors dynasty grows. I’m taking the Golden State Warriors to be #NBAFinals champs. #DubNation Make your pick 👇with @budweiserusa for a chance to win a limited-edition Budweiser Championship Bottle. #sweepstakes,0.8885,0.550000
7,1134248228449411072,"The #NBAFinals have become the norm for #DubNation\nSave $30 on this Ready To Hang 16x20 Stretched Canvas of Klay, Steph &amp; KD! Now $50 in our #YEG Store on #StAlbert Trail. Shop the #ProAmFanCave Online CLICK LINK: https://t.co/z8lDf59Kjn https://t.co/mtsi6jlAKF",0.7177,0.068750
8,1134248241132908549,Coach Kerr knows the energy in Toronto and on #WarriorsGround will be electric this series 🎙️ https://t.co/A6iwyVTVCa,0.2732,0.000000
9,1134248269855502336,"Steph Curry walks into Scotiabank Arena ahead of @warriors/@Raptors Game 1! #StrengthInNumbers \n\n🚨: GSW/TOR, Game 1\n⏰: 9:00pm/et \n🇺🇸: ABC 🇨🇦: Sportsnet https://t.co/NttQJq3NGJ",0.0000,-0.450000


## GSW

In [74]:
df_gsw = pd.DataFrame(gsw["text"])
df_gsw.columns = ["Tweets"]

In [75]:
df_tweet = [0]*len(df_gsw["Tweets"])
token_ls = [0]*len(df_gsw["Tweets"])

for num, text in tqdm(enumerate(df_gsw["Tweets"])):
    token_ls[num] = tokenization(tweets_cleaner(text))

112043it [00:39, 2815.98it/s]


In [76]:
sentence_tokenized = [0]*len(df_gsw["Tweets"])
for num, token in tqdm(enumerate(token_ls)):
    sentence_tokenized[num] = back_to_clean_sent(token)

112043it [00:01, 57617.88it/s]


In [77]:
# Vader
def sentiment_analysis(sentence):
    ris = analyser.polarity_scores(sentence)["compound"]
    return ris
                             
sentiment_token = []
for x in tqdm(sentence_tokenized):
    sentiment_token.append(sentiment_analysis(x))

100%|████████████████████████████████████████████████████████████████████████| 112043/112043 [00:19<00:00, 5789.70it/s]


In [78]:
df_gsw["sentiment"] = sentiment_token

In [81]:
gsw["sentiment"] = df_gsw["sentiment"]

In [83]:
gsw.to_csv("C:\\Users\\fedem\\Desktop\\Università\\DATA_SCIENCE\\I_ANNO\\DATA_MANAGEMENT_AND_DATA_VISUALIZATION\\Progetto\\final_gsw.csv", encoding = "utf-8", index = False)

## Toronto

In [84]:
df_tor = pd.DataFrame(tor["text"])
df_tor.columns = ["Tweets"]

In [85]:
df_tweet = [0]*len(df_tor["Tweets"])
token_ls = [0]*len(df_tor["Tweets"])

for num, text in tqdm(enumerate(df_tor["Tweets"])):
    token_ls[num] = tokenization(tweets_cleaner(text))

569181it [03:43, 2546.44it/s]


In [86]:
sentence_tokenized = [0]*len(df_tor["Tweets"])
for num, token in tqdm(enumerate(token_ls)):
    sentence_tokenized[num] = back_to_clean_sent(token)

569181it [00:12, 45985.36it/s]


In [87]:
# Vader
                              
sentiment_token = []
for x in tqdm(sentence_tokenized):
    sentiment_token.append(sentiment_analysis(x))

100%|████████████████████████████████████████████████████████████████████████| 569181/569181 [01:59<00:00, 4766.87it/s]


In [88]:
df_tor["sentiment"] = sentiment_token

In [89]:
tor["sentiment"] = df_tor["sentiment"]

In [91]:
tor.to_csv("C:\\Users\\fedem\\Desktop\\Università\\DATA_SCIENCE\\I_ANNO\\DATA_MANAGEMENT_AND_DATA_VISUALIZATION\\Progetto\\final_tor.csv", encoding = "utf-8", index = False)

## Neutro

In [92]:
df_neutro = pd.DataFrame(neutro["text"])
df_neutro.columns = ["Tweets"]

In [93]:
df_tweet = [0]*len(df_neutro["Tweets"])
token_ls = [0]*len(df_neutro["Tweets"])

for num, text in tqdm(enumerate(df_neutro["Tweets"])):
    token_ls[num] = tokenization(tweets_cleaner(text))

624613it [04:12, 2471.57it/s]


In [94]:
sentence_tokenized = [0]*len(df_neutro["Tweets"])
for num, token in tqdm(enumerate(token_ls)):
    sentence_tokenized[num] = back_to_clean_sent(token)

624613it [00:16, 38745.33it/s]


In [95]:
# Vader
                              
sentiment_token = []
for x in tqdm(sentence_tokenized):
    sentiment_token.append(sentiment_analysis(x))

100%|████████████████████████████████████████████████████████████████████████| 624613/624613 [02:36<00:00, 4001.52it/s]


In [96]:
df_neutro["sentiment_token"] = sentiment_token

In [97]:
neutro["sentiment"] = df_neutro["sentiment_token"]

In [98]:
neutro

,id,n_tweet,date,time,device,retweet_count,place_tweet,place_user,text,hashtag,...,ASSIST_home,ASSIST_away,PERSONAL_home,PERSONAL_away,TECHNICAL_home,TECHNICAL_away,retweet_id,timestat,quarter,sentiment
0,1134248121817395200,4,2019-05-31,00:00:05,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",0,NaN,"Chicago, IL",My @warriors vs @Raptors preview for @WinnersWhiners:\nhttps://t.co/Ne3zGKqNQl …\n#GSWvsTOR #DubNation #Oakland #SanFrancisco #BayArea #Warriors #WeTheNorth #NBA #NBAFinals #NBAPlayoffs #NBATwitter #NBAPicks #GamblingTwitter #SportsBet #BettingTips #FreePicks #StephenCurry #GSW,"['sanfrancisco', 'oakland', 'nbapicks', 'stephencurry', 'bayarea', 'nbafinals', 'bettingtips', 'gswvstor', 'freepicks', 'nba', 'wethenorth', 'sportsbet', 'warriors', 'dubnation', 'nbaplayoffs', 'gamblingtwitter', 'nbatwitter', 'gsw']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000
1,1134248121901256704,3,2019-05-31,00:00:05,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",0,NaN,United States,Cousins Gives Warriors Another Weapon Tonight \n\n#NBAFinals \n\nhttps://t.co/20xPjawJrL,['nbafinals'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.2960
2,1134248122010353665,6,2019-05-31,00:00:05,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",0,NaN,"Schenectady, N.Y.","#PartingSchotts Podcast: Talking @FreihofersRun with @DDNukuri, @jalenpickett2's decision with @ByMichaelKelly, #NBAFinals with @GuilderlandCSD grad @marckestecher and @Albany_Empire with https://t.co/QCJJqrVEVj @SienaMBB #NBAPlayoffs @OfficialAFL @JawsCEOQB https://t.co/ZyNXghpXi3","['nbaplayoffs', 'partingschotts', 'nbafinals']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000
3,1134248122727718913,9,2019-05-31,00:00:05,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",0,NaN,NaN,"It's almost game time as Raptors make their first NBA championship appearance against two-time defending champions, Golden State Warriors.\n\nCheck the latest #NBAFinals odds here: https://t.co/JnDAzGrCVS\n\n#WeTheNorth | #StrengthInNumbers \n\n#NBA https://t.co/dlsjkFw9o9","['strengthinnumbers', 'nbafinals', 'wethenorth', 'nba']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.7430
4,1134248122794856450,12,2019-05-31,00:00:05,"<a href=""https://studio.twitter.com"" rel=""nofollow"">Twitter Media Studio</a>",0,NaN,"Oakland, CA","#NBAFinals, Game 1 -- presented by @googlenest \n\n🏀 Warriors vs. Raptors\n🕓 6:00 PM \n📺 #NBAonABC\n📻 @957thegame &amp; @ESPNRadio \n📱 https://t.co/ZkjPJj14fy \n📝 https://t.co/1xDGyr71k1 https://t.co/bN3hn7WsnL","['nbafinals', 'nbaonabc']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000
5,1134248122933227521,7,2019-05-31,00:00:05,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",0,NaN,"Sioux Falls, SD",It's time! Every game and every moment of the #NBAFinals is right here on #ESPN991. Pregame coverage of Game 1 starts right now with the game beginning around 8:00.,"['espn991', 'nbafinals']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000
6,1134248123004407808,8,2019-05-31,00:00:05,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",0,NaN,NaN,The #NBAFinals is here. Perfect timing for another #NitroGameThread 🙌\n\n#DubNation 🆚 #WeTheNorth\n\nDiscuss Game 1 for a chance to win BTC* 💰\n\nRT this post &amp; REPLY with #NitroGameThread + thoughts about the game + tag two friends\n\nSubmit Nitro ID ➡ https://t.co/Py2t8oljSx https://t.co/z...,"['nitrogamethread', 'dubnation', 'nbafinals', 'wethenorth']",...,NaN,NaN,NaN,NaN,NaN,NaN,1.133984e+18,NaN,0,0.9118
7,1134248123050602497,14,2019-05-31,00:00:05,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",0,NaN,"Schenectady, NY","#PartingSchotts Podcast: Talking @FreihofersRun with @DDNukuri, @jalenpickett2's decision with @ByMichaelKelly, #NBAFinals with @GuilderlandCSD grad @marckestecher and @Albany_Empire wit

In [99]:
neutro.to_csv("C:\\Users\\fedem\\Desktop\\Università\\DATA_SCIENCE\\I_ANNO\\DATA_MANAGEMENT_AND_DATA_VISUALIZATION\\Progetto\\final_neutro.csv", encoding = "utf-8", index = False)